# Showcase the EarthDaily Simulated dataset

This notebook illustrates how to use the EarthDaily Simulated dataset to generate a field level datacube that will use to generate a vegetation index evolution animation highlighting new capabilities enabled by daily imagery.

For this example the data cube has been generated using https://github.com/earthdaily/reflectance-datacube-processor with the following parameters:
- Geometry: "POLYGON ((-96.444476707042 41.18311650256777, -96.4347884854818 41.18295356510254, -96.43489673376754 41.176069086814266, -96.4445308311847 41.176028348103074, -96.44436845875661 41.17879852277147, -96.444476707042 41.18311650256777))"
- Collection: earthdaily-simulated-cloudless-l2a-cog-edagro
- Dates: 2019-05-01 to 2019-05-31
- Assets: red, green, blue

In [1]:
import sys
import os
sys.path.append(os.path.join('..','src'))
from dotenv import load_dotenv
load_dotenv()
import xarray as xr
from geogif import dgif
from utils.utils import open_datacube
from cloudpathlib import S3Path, AzureBlobClient, AzureBlobPath

## 1️⃣ Open EarthDaily simulated dataset

- Load Datacube from AWS

In [3]:
image = S3Path("s3://" + os.getenv("AWS_BUCKET_NAME") + "/2019-05-01_2019-05-31_ttestEDataset_earthdaily-simulated-cloudless-l2a-cog-edagro_datacube.zarr")
cube = open_datacube(image)

- Load Datacube from Azure Cloud Storage

In [4]:
# Init Azure account/credentials
account_url = f"https://{os.environ['AZURE_ACCOUNT_NAME']}.blob.core.windows.net"
az_client = AzureBlobClient(account_url, credential = os.environ['AZURE_SAS_CREDENTIAL'])

In [5]:
# Load datacube with the Agro Cloud Mask
uri = "az://" + os.getenv("AZURE_BLOB_CONTAINER_NAME") + "/2019-05-01_2019-05-31_ttestEDataset_earthdaily-simulated-cloudless-l2a-cog-edagro_datacube.zarr"
path = AzureBlobPath(uri,client=az_client)
cube = open_datacube(path)

In [6]:
cube

<xarray.Dataset> Size: 397kB
Dimensions:      (time: 30, y: 22, x: 25)
Coordinates:
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 240B 2019-05-01T10:42:14 ... 2019-05-3...
  * x            (x) float64 200B 3.592e+05 3.592e+05 ... 3.594e+05 3.594e+05
  * y            (y) float64 176B 4.81e+06 4.81e+06 ... 4.81e+06 4.81e+06
Data variables:
    blue         (time, y, x) float64 132kB dask.array<chunksize=(1, 22, 25), meta=np.ndarray>
    green        (time, y, x) float64 132kB dask.array<chunksize=(1, 22, 25), meta=np.ndarray>
    red          (time, y, x) float64 132kB dask.array<chunksize=(1, 22, 25), meta=np.ndarray>

 ## 2️⃣ Plot one year of data

In [7]:
datacube = cube[["red", "green", "blue"]].to_array()/10000
datacube = datacube.rename({'variable':'band'})
datacube = datacube.transpose('time','band','y','x')

In [8]:
result_animated = dgif(datacube,date_format=None).compute()
scale = 20
result_animated.height = int(datacube.shape[3] * scale)
result_animated.width = int(datacube.shape[2] * scale)

c:\Users\kls\AppData\Local\anaconda3\envs\reflectance-processor\Lib\site-packages\geogif\gif.py:190: RuntimeWarning: invalid value encountered in cast
  u8 = (data * 255).astype("uint8")


In [9]:
print('Animation of the EarthDaily Simulated dataset between May 1st 2019 and May 29st 2020')
result_animated

Animation of the EarthDaily Simulated dataset between May 1st 2019 and May 29st 2020
